In [1]:
    base = 'gdrive/MyDrive/ppat/chi_train/'

#Runtime Info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


Sat Dec  3 21:42:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#setup
from google.colab import drive
drive.mount('/content/gdrive')
!rm -rf dataset
!unzip {base}dataset.zip>/dev/null
!pip install tensorflow scikit-learn imbalanced-learn pandas opencv-python-headless


Mounted at /content/gdrive
Archive:  gdrive/MyDrive/ppat/chi_train/dataset.zip
   creating: dataset/
  inflating: dataset/knob0353.png    
  inflating: dataset/lpf_notch_on.png  
  inflating: dataset/moog0054.png    
  inflating: dataset/ButtonEnableActive_2x1.png  
  inflating: dataset/BackgroundMono.png  
  inflating: dataset/Needle_059.png  
  inflating: dataset/HDR_BTN_Browser.png  
  inflating: dataset/Panel Perf OP-OP.png  
  inflating: dataset/LA2A00026.png   
  inflating: dataset/knob_small1101.png  
  inflating: dataset/knob00126.png   
  inflating: dataset/window_38.png   
  inflating: dataset/knob0151.png    
  inflating: dataset/IOMeterSwitch_InputOn.png  
  inflating: dataset/Meters_LCR.png  
  inflating: dataset/knob0145.png    
  inflating: dataset/Prefs_Dropdown_Colors_Normal.png  
  inflating: dataset/window_04.png   
  inflating: dataset/menu19.png      
  inflating: dataset/BRWS_BTN_FooterEdit1.png  
  inflating: dataset/Large_Knob033.png  
  inflating: dataset/knob_

In [3]:
import numpy as np
import random as rn
import tensorflow as tf
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(0)
rn.seed(0)
tf.random.set_seed(0)
from datetime import datetime
from glob import glob
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from random import randint
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.layers import Dense, add, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, Input, Concatenate, Cropping2D, ActivityRegularization, RandomBrightness, RandomContrast, RandomCrop, RandomFlip, RandomHeight, RandomRotation, RandomTranslation, RandomWidth, RandomZoom
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
import codecs
import cv2
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
import time
from google.colab import runtime
tf.get_logger().setLevel('ERROR')

In [4]:
# Util functions
def log(message):
	print(message)
	file = codecs.open(base+"log.txt", "a", "cp1252", "replace")
	print(message, file=file)

def load(file):
	im = cv2.imread(file)
	if im is None:
		print("Can't load", file)
		return
	return tf.image.resize(im, size[:2])


In [5]:
class DataGenerator(Sequence):
	def __init__(self,x, y=None, batch_size = 1, mode="train", alpha=0.2):
		self.x = x
		self.y = y
		self.mode = mode
		self.alpha = alpha
		self.batch_size = batch_size
		self.img_gen = ImageDataGenerator(zca_whitening=False, rotation_range=20, shear_range=0.2, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True)
		self.on_epoch_end()

	def __len__(self):
		return int(np.ceil(len(self.x) / float(self.batch_size)))

	def on_epoch_end(self):
		self.indexes = np.arange(len(self.x))
		if self.mode == "train": np.random.shuffle(self.indexes)

	def aug(self, x):
		if self.mode == "train" and randint(1,10)>1: x = self.img_gen.random_transform(x)
		return x

	def __getitem__(self, index):
		indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
		x_batch = [imgs[x] for x in self.x[indexes]]
		x_batch = np.array(x_batch)
		#x_batch = self.x[indexes]
		if self.y is not None: y_batch = self.y[indexes]
		#if self.mode == "train": x_batch, y_batch = self.mixup(x_batch, y_batch)
		if self.y is None: return x_batch
		return x_batch, y_batch

	def mixup(self, x, y):
		n = x.shape[0]
		l = np.random.beta(self.alpha, self.alpha, n)
		x_l = l.reshape(n, 1, 1, 1)
		y_l = l.reshape(n, 1)
		#index_l = np.arange(n)
		#np.random.shuffle(index_l)

		x1 = x
		x2 = x[::-1]
		x = x1 * x_l + x2 * (1 - x_l)
		
		y1 = y
		y2 = y[::-1]
		y = y1 * y_l + y2 * (1 - y_l)
		return x, y

	def mixup2(self, x, y):
		n = x.shape[0]
		l = np.random.beta(self.alpha, self.alpha)
		
		xl = x[::-1]
		x = x * l + xl * (1 - l)
		
		yl = y[::-1]
		y = y * l + yl * (1 - l)
		return x, y


In [6]:
class MetricCallback(Callback):
	def __init__(self, x, y, k=0, patience=0, restore_best=True):
		super().__init__()
		self.x = x
		self.y = y
		self.k=k
		self.best_score = 0.0
		self.best_epoch = 0
		self.best_weights = None
		self.patience = patience
		self.restore_best = restore_best

	def restore(self):
		self.model.set_weights(self.best_weights)

	def on_epoch_end(self, epoch, logs={}):
		pred_gen= DataGenerator(self.x, mode="predict")
		preds = self.model.predict(pred_gen, verbose=0)
		final = f1_score(self.y.argmax(-1), preds.argmax(-1), average='micro')
		if final>self.best_score:
			for f in glob("*.h5"): os.remove(f)
			self.model.save(f"model {self.k+1} {epoch+1} f{final:.3f}.h5")
			!rm {base}*.h5
			!cp *.h5 {base}
			log(f"{epoch+1} Best Score: improved from {self.best_score:.3f} to {final:.3f}")
			self.best_score = final
			self.best_epoch = epoch
			self.best_weights = self.model.get_weights()
		else: log(f"{epoch+1} Score: {final:.3f}, Best epoch: {self.best_epoch+1}, {self.best_score:.3f}")

		if self.patience>0 and epoch-self.best_epoch >= self.patience:
			print("Stopping...")
			self.stopped_epoch = epoch
			self.model.stop_training = True
			if self.restore_best: self.restore()
			return

		if not epoch == self.best_epoch and (epoch-self.best_epoch)%int(self.patience/2) == 0:
			old_lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
			new_lr = old_lr * 0.1
			tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
			self.restore()
			log(f"Reducing learning rate from {old_lr} to {new_lr}")


In [7]:
def create_model(input_shape, output_shape):
	inputs = Input(shape=input_shape)
	x = RandomBrightness(0.2)(inputs)
	x = RandomContrast(0.2)(x)
	x = RandomFlip()(x)
	x = RandomRotation(0.2)(x)
	x = RandomTranslation(0.2, 0.2)(x)
	x = RandomZoom(0.2, 0.2)(x)
	base_model = MobileNetV3Small(input_tensor=x, include_top=False, weights='imagenet', pooling="avg")
	for layer in base_model.layers[8:-1]: layer.trainable = False
	x = Dense(256)(base_model.output)
	x = BatchNormalization()(x)
	x = Activation("relu")(x)
	x = Dropout(0.2)(x)
	outputs =Dense(output_shape,activation='softmax')(base_model.output)
	model=Model(inputs=inputs, outputs=outputs)
	model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy',metrics=['accuracy'])
	for i, layer in enumerate(model.layers):
		if layer.trainable: print(i, layer.name, "Trainable:", layer.trainable, "Input:", layer.input_shape, "Output:", layer.output_shape)
	#print(model.summary())
	return model


In [8]:
#Imblearn
def balance(x, y):
	print("Balancing samples...")
	ros = RandomOverSampler()
	x, y = ros.fit_resample(x.reshape(-1,1), y)
	# Can't use synthetic for memory limitation
	#smote_enn = SMOTEENN(random_state=0)
	#x, y = smote_enn.fit_resample(x, y)
	x = x.reshape(-1)
	u, c = np.unique(y, return_counts=True)
	names = [le.classes_[l] for l in u]
	dist = list(zip(names, c))
	dist = sorted(dist, key = lambda x: x[1])
	print("Distribution after over sampling:", dist)
	return x, y
	

In [9]:
size = (224, 224, 3)
batch_size = 32
k=0
t_start = time.time()
now = datetime.now()
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
log(f"Starting at: {date_time}")
log(f"Size: {size}, batch_size: {batch_size}, k_fold: {k}")
log(f"Tensorflow: {tf.version.VERSION}")
log(f"SKLearn: {sklearn.__version__}")


Starting at: 12/03/2022, 21:45:15
Size: (224, 224, 3), batch_size: 32, k_fold: 0
Tensorflow: 2.9.2
SKLearn: 1.0.2


In [11]:
#Load and cleanup
df = pd. read_csv (base+"label.csv")
print(df['label'].value_counts())
search = df['label'].value_counts().reset_index(name="count").query("count < 6")["index"]
print("Removing", search.tolist())
df = df[~df['label'].isin(search)]
df = df.dropna(subset=['label'])
print(df['label'].value_counts())

files = df['image']
files = ["dataset/"+file for file in files]
labels = df['label']
le = LabelEncoder()
labels = le.fit_transform(labels)
bad = []
imgs = {}
for i in range(len(files)):
	data = load(files[i])
	if data is None:
		bad.append(i)
		print("Can't read: ", files[i])
	else: imgs[files[i]] = data
x = np.delete(np.array(files), np.array(bad))
y = np.delete(np.array(labels), np.array(bad))
x = np.array(x)


knob                 679
non-interactive      389
remove               293
needle               173
icon                  64
button                53
radio button          35
multiple knobs        31
dropdown              29
multiple elements     25
meter                 24
slider                20
light                 16
arrow                 15
switch                12
unknown               10
other                  8
multiple buttons       8
music note             7
multiple switches      4
confused               2
graph                  1
fader                  1
wheel                  1
scale                  1
Name: label, dtype: int64
Removing ['multiple switches', 'confused', 'graph', 'fader', 'wheel', 'scale']
knob                 679
non-interactive      389
remove               293
needle               173
icon                  64
button                53
radio button          35
multiple knobs        31
dropdown              29
multiple elements     25
meter               

In [12]:
#Divide train and test set
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1)
splits = list(sss.split(np.zeros(y.shape[0]), y))
train_index, test_index = splits[k]
x_train, y_train = x[train_index], y[train_index]
x_test, y_test = x[test_index], y[test_index]
log(f"x_train: {x_train.shape}, y_train:{y_train.shape}, x_test: {x_test.shape}, y_test: {y_test.shape}")
x_train, y_train = balance(x_train, y_train)
y_train= to_categorical(y_train)
y_test = to_categorical(y_test)
log(f"x_train: {x_train.shape}, y_train:{y_train.shape}, x_test: {x_test.shape}, y_test: {y_test.shape}")


x_train: (1701,), y_train:(1701,), x_test: (189,), y_test: (189,)
Balancing samples...
Distribution after over sampling: [('arrow', 611), ('button', 611), ('dropdown', 611), ('icon', 611), ('knob', 611), ('light', 611), ('meter', 611), ('multiple buttons', 611), ('multiple elements', 611), ('multiple knobs', 611), ('music note', 611), ('needle', 611), ('non-interactive', 611), ('other', 611), ('radio button', 611), ('remove', 611), ('slider', 611), ('switch', 611), ('unknown', 611)]
x_train: (11609,), y_train:(11609, 19), x_test: (189,), y_test: (189, 19)


In [ ]:
#Train
log(f"Training fold {k}...")
model = create_model(size, y_train.shape[1])
train_gen= DataGenerator(x_train, y_train, batch_size=batch_size)
val_gen= DataGenerator(x_test, y_test, batch_size=batch_size, mode="val")
mc = MetricCallback(x_test, y_test, patience=50)
logger = CSVLogger(base+'train.csv')
callbacks = [mc, logger]
history = model.fit(train_gen, validation_data=val_gen, epochs=1000, callbacks =callbacks, verbose=2) #, workers=multiprocessing.cpu_count(), use_multiprocessing=True)
#End
now = datetime.now()
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
log(f"Ending at: {date_time}")
t_finish = time.time()
total_time = (t_finish-t_start)/60
log(f"It took {total_time} minutes")


Training fold 0...
4334752/4334752 [==============================] - 0s 0us/step
0 input_1 Trainable: True Input: [(None, 224, 224, 3)] Output: [(None, 224, 224, 3)]
1 random_brightness Trainable: True Input: (None, 224, 224, 3) Output: (None, 224, 224, 3)
2 random_contrast Trainable: True Input: (None, 224, 224, 3) Output: (None, 224, 224, 3)
3 random_flip Trainable: True Input: (None, 224, 224, 3) Output: (None, 224, 224, 3)
4 random_rotation Trainable: True Input: (None, 224, 224, 3) Output: (None, 224, 224, 3)
5 random_translation Trainable: True Input: (None, 224, 224, 3) Output: (None, 224, 224, 3)
6 random_zoom Trainable: True Input: (None, 224, 224, 3) Output: (None, 224, 224, 3)
7 rescaling Trainable: True Input: (None, 224, 224, 3) Output: (None, 224, 224, 3)
235 avg_pool Trainable: True Input: (None, 7, 7, 576) Output: (None, 576)
236 dense_1 Trainable: True Input: (None, 576) Output: (None, 19)
Epoch 1/1000


In [ ]:
runtime.unassign()